# silver_comercio_ext_indices.tb_referencia_ncm_unidade
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_UNIDADE_delta`
## 📌 Descrição do arquivo

Tabela de referência das **unidades estatísticas** usadas na `QT_ESTAT`.

| Coluna    | Descrição técnica | Interpretação humana                           |
| --------- | ----------------- | ---------------------------------------------- |
| `CO_UNID` | Código da unidade | Identificador usado em fatos e na dimensão NCM |
| `NO_UNID` | Nome da unidade   | Nome por extenso da unidade                    |
| `SG_UNID` | Sigla da unidade  | Abreviação usada em relatórios                 |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_UNIDADE_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_unidade/"   
val silverTable =    "tb_referencia_ncm_unidade"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_UNID" , StringType ,  nullable = false ) ,
      StructField ( "NO_UNID" , StringType ,  nullable = false ) ,
      StructField ( "SG_UNID" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_UNID", trim(upper(col("CO_UNID").cast(StringType))))
  .withColumn("NO_UNID", col("NO_UNID").cast(StringType))
  .withColumn("SG_UNID", trim(upper(col("SG_UNID").cast(StringType))))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NCM_UNIDADE_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_UNID").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_UNID"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_UNID"), col("NO_UNID"), col("SG_UNID"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}